fix ballhandlers being out of bounds, add a gaussian

In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [2]:
# with open("id_to_player.json", "w") as f:
#     json.dump(id_to_player, f)
# with open("player_to_id.json", "w") as f:
#     json.dump(player_to_id, f)

with open("id_to_player.json") as f:
    for l in f:
        id_to_player = json.loads(l)
id_to_player = {int(k): v for (k, v) in id_to_player.items()}
with open("player_to_id.json") as f:
    for l in f:
        player_to_id = json.loads(l)
player_to_id = {k: int(v) for (k, v) in player_to_id.items()}

In [3]:
#id_to_team = {}
# team_to_id = {}
# for root, dirs, files in os.walk(".\data"):
#     if len(id_to_team) == 30:
#         break
#     for fil in files:
#         with open(os.path.join(root, fil)) as f:
#             for l in f:
#                 line = json.loads(l)
#                 home_id = line["events"][0]["home"]["teamid"]
#                 home_name = line["events"][0]["home"]["name"]
#                 away_id = line["events"][0]["visitor"]["teamid"]
#                 away_name = line["events"][0]["visitor"]["name"]
#                 id_to_team[home_id] = home_name
#                 id_to_team[away_id] = away_name
#                 team_to_id[home_name] = home_id
#                 team_to_id[away_name] = away_id
#                 break

# with open("id_to_team.json", "w") as f:
#     json.dump(id_to_team, f)
# with open("team_to_id.json", "w") as f:
#     json.dump(team_to_id, f)

with open("id_to_team.json") as f:
    for l in f:
        id_to_team = json.loads(l)
id_to_team = {int(k): v for (k, v) in id_to_team.items()}        
with open("team_to_id.json") as f:
    for l in f:
        team_to_id = json.loads(l)
team_to_id = {k: int(v) for (k, v) in team_to_id.items()}

In [4]:
def distance(x1, y1, x2, y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

In [5]:
def get_zone(x, y):
    if x < 0 or x > 94:
        return 0
    elif 19 <= y <= 31 and (x < 19 or x > 75):
        return 1 
    elif (0 <= x <= 14 or 80 <= x <= 94) and 0 <= y <= 3:
        return 2
    elif (0 <= x <= 14 or 80 <= x <= 94) and 47 <= y <= 50:
        return 3
    elif (0 <= x <= 14 or 80 <= x <= 94) and 3 < y < 19:
        return 4
    elif (0 <= x <= 14 or 80 <= x <= 94) and 31 < y < 47:
        return 5
    elif ((x < 47 and (x-5.25)**2 + (y-25)**2 < 23.75**2) or (x >= 47 and (x-88.75)**2 + (y-25)**2 < 23.75**2)) and y <= 25:
        return 6
    elif ((x < 47 and (x-5.25)**2 + (y-25)**2 < 23.75**2) or (x >= 47 and (x-88.75)**2 + (y-25)**2 < 23.75**2)) and y > 25:
        return 7
    elif y <= 25:
        return 8
    else:
        return 9

In [6]:
def transitions(abbrev, team_name):
    team_id = team_to_id[team_name]
    transition_dict = {}
    seen_moments = set()

    for root, dirs, files in os.walk(".\data"):
        if abbrev in root:
            print root
            for fil in files:
                with open(os.path.join(root, fil)) as f:
                    for l in f:
                        line = json.loads(l)
                        for i in range(len(line["events"])):
                            for mom in line["events"][i]["moments"]:
                                mom_tuple = (mom[2], mom[3], mom[5][0][2], mom[5][0][3])
                                if mom_tuple not in seen_moments:
                                    for val in mom[5]:
                                        player = val[1]
                                        team = val[0]
                                        if team == team_id and player != -1:
                                            zone = get_zone(val[2], val[3])
                                            if team not in transition_dict:
                                                transition_dict[team] = {}
                                            if player not in transition_dict[team]:
                                                transition_dict[team][player] = [np.zeros(shape=(10, 10)), zone]
                                            else:
                                                if transition_dict[team][player][1] != zone:
                                                    transition_dict[team][player][0][transition_dict[team][player][1], zone] += 1
                                            transition_dict[team][player][1] = zone
                                    seen_moments.add(mom_tuple)
    return transition_dict

In [7]:
def distances(abbrev, team_name):
    moment_dists = []
    team_id = team_to_id[team_name]
    seen_moments = set()

    for root, dirs, files in os.walk(".\data"):
        if abbrev in root:
            print root
            for fil in files:
                with open(os.path.join(root, fil)) as f:
                    for l in f:
                        line = json.loads(l)
                        for i in range(len(line["events"])):
                            for mom in line["events"][i]["moments"]:
                                mom_tuple = (mom[2], mom[3], mom[5][0][2], mom[5][0][3])
                                if mom_tuple not in seen_moments:
                                    ballx = mom[5][0][2]
                                    bally = mom[5][0][3]
                                    dists = []
                                    for val in mom[5]:
                                        team = val[0]
                                        player = val[1]
                                        if player != -1 and team == team_id:
                                            dists.append((id_to_player[player], distance(val[2], val[3], ballx, bally)))
                                    seen_moments.add(mom_tuple)
                                    if dists:
                                        moment_dists.append(sorted(dists, key=lambda x:x[1]))
    return moment_dists

In [18]:
def get_ballhandlers(moment_dists):
    ballhandlers = []
    for i, mom in enumerate(moment_dists):
        if mom[0][1] > 3:
            ballhandler = None
            for j in range(i-1, i-5, -1):
                if j < 0:
                    break
                if ballhandlers[j][0] is None:
                    for ind in range(j, i):
                        ballhandlers[ind] = (None, ballhandlers[ind][1])
        else:
            if i == 0 or i > 0 and mom[0][0] == moment_dists[i-1][0][0]:
                ballhandler = mom[0][0]
                if i > 0 and ballhandlers[i-1][0] is None:
                    for j in range(i-1, i-5, -1):
                        if j < 0:
                            break
                        if ballhandlers[j][0] == ballhandler:
                            for ind in range(j, i):
                                ballhandlers[ind] = (ballhandler, ballhandlers[ind][1])
            else:
                ballhandler = None
        ballhandlers.append((ballhandler, mom[0][1]))
    return ballhandlers

In [21]:
def ballhandler_transitions(abbrev, team_name, ballhandlers):
    team_id = team_to_id[team_name]
    b_transition_dict = {}
    seen_moments = set()

    moment_num = 0
    for root, dirs, files in os.walk(".\data"):
        if abbrev in root:
            print root
            for fil in files:
                with open(os.path.join(root, fil)) as f:
                    for l in f:
                        line = json.loads(l)
                        for i in range(len(line["events"])):
                            last_bh = None
                            for mom in line["events"][i]["moments"]:
                                mom_tuple = (mom[2], mom[3], mom[5][0][2], mom[5][0][3])
                                if mom_tuple not in seen_moments:
                                    for val in mom[5]:
                                        team = val[0]
                                        player = val[1]
                                        if moment_num >= len(ballhandlers):
                                            return b_transition_dict
                                        if team == team_id and player != -1 and id_to_player[player] == ballhandlers[moment_num][0]:
                                            zone = get_zone(val[2], val[3])
#                                             if zone == 0:
#                                                 print val[2], val[3], id_to_player[player]
                                            if team not in b_transition_dict:
                                                b_transition_dict[team] = {}
                                            if player not in b_transition_dict[team]:
                                                b_transition_dict[team][player] = [np.zeros(shape=(11, 11)), zone]
                                            else:
                                                if b_transition_dict[team][player][1] != zone:
                                                    b_transition_dict[team][player][0][b_transition_dict[team][player][1], zone] += 1
                                            b_transition_dict[team][player][1] = zone
                                        elif team == team_id and player != -1 and ballhandlers[moment_num][0] is None and last_bh is not None:
                                            player = player_to_id[last_bh]
                                            b_transition_dict[team][player][0][b_transition_dict[team][player][1], 10] += 1
                                    last_bh = ballhandlers[moment_num][0]       
                                    moment_num += 1
                                    seen_moments.add(mom_tuple)
    return b_transition_dict

In [ ]:
trans = transitions("GSW", "Golden State Warriors")

In [10]:
dists = distances("GSW", "Golden State Warriors")

.\data\01.02.2016.DEN.at.GSW
.\data\01.04.2016.CHA.at.GSW
.\data\01.05.2016.GSW.at.LAL
.\data\01.08.2016.GSW.at.POR
.\data\01.09.2016.GSW.at.SAC
.\data\01.11.2016.MIA.at.GSW
.\data\01.13.2016.GSW.at.DEN
.\data\01.14.2016.LAL.at.GSW
.\data\01.18.2016.GSW.at.CLE
.\data\01.20.2016.GSW.at.CHI
.\data\01.22.2016.IND.at.GSW
.\data\10.27.2015.NOP.at.GSW
.\data\10.31.2015.GSW.at.NOP
.\data\11.02.2015.MEM.at.GSW
.\data\11.04.2015.LAC.at.GSW
.\data\11.06.2015.DEN.at.GSW
.\data\11.07.2015.GSW.at.SAC
.\data\11.09.2015.DET.at.GSW
.\data\11.11.2015.GSW.at.MEM
.\data\11.12.2015.GSW.at.MIN
.\data\11.14.2015.BKN.at.GSW
.\data\11.17.2015.TOR.at.GSW
.\data\11.19.2015.GSW.at.LAC
.\data\11.20.2015.CHI.at.GSW
.\data\11.22.2015.GSW.at.DEN
.\data\11.24.2015.LAL.at.GSW
.\data\11.27.2015.GSW.at.PHX
.\data\11.28.2015.SAC.at.GSW
.\data\11.30.2015.GSW.at.UTA
.\data\12.02.2015.GSW.at.CHA
.\data\12.05.2015.GSW.at.TOR
.\data\12.06.2015.GSW.at.BKN
.\data\12.08.2015.GSW.at.IND
.\data\12.11.2015.GSW.at.BOS
.\data\12.12.2

In [19]:
bh = get_ballhandlers(dists)

In [23]:
bh_trans = ballhandler_transitions("GSW", "Golden State Warriors", bh)

.\data\01.02.2016.DEN.at.GSW
.\data\01.04.2016.CHA.at.GSW
.\data\01.05.2016.GSW.at.LAL
.\data\01.08.2016.GSW.at.POR
.\data\01.09.2016.GSW.at.SAC
.\data\01.11.2016.MIA.at.GSW
.\data\01.13.2016.GSW.at.DEN
.\data\01.14.2016.LAL.at.GSW
.\data\01.18.2016.GSW.at.CLE
.\data\01.20.2016.GSW.at.CHI
.\data\01.22.2016.IND.at.GSW
.\data\10.27.2015.NOP.at.GSW
.\data\10.31.2015.GSW.at.NOP
.\data\11.02.2015.MEM.at.GSW
.\data\11.04.2015.LAC.at.GSW
.\data\11.06.2015.DEN.at.GSW
.\data\11.07.2015.GSW.at.SAC
.\data\11.09.2015.DET.at.GSW
.\data\11.11.2015.GSW.at.MEM
.\data\11.12.2015.GSW.at.MIN
.\data\11.14.2015.BKN.at.GSW
.\data\11.17.2015.TOR.at.GSW
.\data\11.19.2015.GSW.at.LAC
.\data\11.20.2015.CHI.at.GSW
.\data\11.22.2015.GSW.at.DEN
.\data\11.24.2015.LAL.at.GSW
.\data\11.27.2015.GSW.at.PHX
.\data\11.28.2015.SAC.at.GSW
.\data\11.30.2015.GSW.at.UTA
.\data\12.02.2015.GSW.at.CHA
.\data\12.05.2015.GSW.at.TOR
.\data\12.06.2015.GSW.at.BKN
.\data\12.08.2015.GSW.at.IND
.\data\12.11.2015.GSW.at.BOS
.\data\12.12.2

In [28]:
for team, players in bh_trans.items():
    for p, trans in players.items():
        print id_to_player[p]
        print np.sum(np.sum(trans, axis=0))
        print normalize(trans[0], axis=1, norm="l1")

Festus Ezeli
17696.0
[[  0.00000000e+00   1.47540984e-01   0.00000000e+00   0.00000000e+00
    1.63934426e-02   8.19672131e-03   3.27868852e-02   1.63934426e-02
    2.45901639e-02   1.63934426e-02   7.37704918e-01]
 [  3.24254215e-03   0.00000000e+00   1.62127108e-04   1.62127108e-04
    1.91309987e-02   1.92931258e-02   3.50194553e-02   3.29118029e-02
    1.31322957e-02   1.52399481e-02   8.61705577e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   7.69230769e-02   0.00000000e+00   0.00000000e+00
    1.53846154e-01   0.00000000e+00   7.69230769e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   3.33333333e-02   3.33333333e-02   3.33333333e-02
    0.00000000e+00   6.66666667e-02   8.33333333e-01]
 [  2.25056264e-03   9.75243811e-02   1.50037509e-03   0.00000000e+00
    0.00000000e+00   6.75168792e-03   3.45086272e-02   1.95048762e-02
    1.87546887e-02   9.00225056e-03   8.10202551e-01]
 [  2.978